In [ ]:
!pip install face-recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=59b45843a1bace1ab4cd14252ea42928cab8f4daf4741625b159263ad15b9d14
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import os
import cv2
import face_recognition
import numpy as np
import h5py

In [ ]:
def load_known_encodings(h5_file, known_faces_dir):
    if os.path.exists(h5_file):
        with h5py.File(h5_file, 'r') as f:
            known_encodings = list(f['encodings'])
            known_names = [name.decode('utf-8') for name in f['names']]
    else:
        known_encodings = []
        known_names = []
        for filename in os.listdir(known_faces_dir):
            file_path = os.path.join(known_faces_dir, filename)
            if os.path.isfile(file_path):
                img = cv2.imread(file_path)
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                encodings = face_recognition.face_encodings(img_rgb)
                if encodings:
                    known_encodings.append(encodings[0])
                    known_names.append(os.path.splitext(filename)[0])
    return known_encodings, known_names

In [ ]:
def save_known_encodings(h5_file, known_encodings, known_names):
    with h5py.File(h5_file, 'w') as f:
        f.create_dataset('encodings', data=known_encodings)
        f.create_dataset('names', data=np.array(known_names, dtype='S'))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

def recognize_faces(image_path, known_encodings, known_names, tolerance=0.6):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to load image {image_path}")
        return "Unknown", None

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(img_rgb)
    face_encodings = face_recognition.face_encodings(img_rgb, face_locations)

    name = "Unknown"
    for face_encoding in face_encodings:
        distances = face_recognition.face_distance(known_encodings, face_encoding)
        best_match_index = np.argmin(distances)
        if distances[best_match_index] < tolerance:
            name = known_names[best_match_index]
            break

    return name, face_encodings

In [ ]:

def main():
    known_faces_dir = '/content/drive/My Drive/known_faces'
    h5_file = 'encodings.h5'
    image_to_recognize = 'a.jpg'  # Change this to the path of the image you want to recognize

    # Load known face encodings and names
    known_encodings, known_names = load_known_encodings(h5_file, known_faces_dir)

    # Perform face recognition on the new image
    recognized_name, face_encodings = recognize_faces(image_to_recognize, known_encodings, known_names)

    # If the face is not recognized, add it to the known encodings with a unique name based on the filename
    if recognized_name == "Unknown" and face_encodings:
        known_encodings.append(face_encodings[0])
        new_name = os.path.splitext(os.path.basename(image_to_recognize))[0]
        known_names.append(new_name)
        save_known_encodings(h5_file, known_encodings, known_names)
        recognized_name = new_name

    print(f"Recognized Name: {recognized_name}")

if __name__ == "__main__":
    main()


Recognized Name: a


Mounted at /content/drive
